In [6]:
url = 'https://projects.fivethirtyeight.com/biden-approval-rating/polls.json'
response = requests.get(url)
if response.status_code == 200:
        data = response.json() 
        crimerates2021newcastle.append(len(data))

NameError: name 'requests' is not defined

In [10]:
import requests

# Replace 'YOUR_API_KEY' with your actual API key

url = 'https://projects.fivethirtyeight.com/biden-approval-rating/polls.json'

# Make a GET request
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()
else:
    print(f"Error: {response.status_code}")
    print(response.text)


In [11]:

#i used chatgbt to sort data here 

Biden_Opinion_Polls = [{'startDate': poll['startDate'], 'approve': poll['answers'][0]['pct'], 'disapprove': poll['answers'][1]['pct']} for poll in data]

    

In [12]:
import requests
import pandas as pd

df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('output.csv', index=False)




In [13]:
import pandas as pd

Biden_Opinion_Polls= [{'startDate': poll['startDate'], 'approve': poll['answers'][0]['pct'], 'disapprove': poll['answers'][1]['pct']} for poll in data]

Biden_dataframe= pd.DataFrame(Biden_Opinion_Polls)

Biden_dataframe.to_csv('Biden_Opinion_Polls_df.csv', index=False)


In [16]:
pip install statsmodels


Note: you may need to restart the kernel to use updated packages.


In [34]:
import altair as alt
import pandas as pd
import statsmodels.api as sm

# Assume 'Biden_dataframe' is your DataFrame
df_long = pd.melt(Biden_dataframe, id_vars=['startDate'], value_vars=['approve', 'disapprove'],
                  var_name='Opinion', value_name='Percentage')

# Altair Chart for points with filled circles
chart_points = alt.Chart(df_long).mark_point(size=50, opacity=0.25, filled=True).encode(
    x=alt.X('startDate:T', axis=alt.Axis(tickCount='month', format='%b %Y', labelAngle=-90)),
    y=alt.Y('Percentage:Q', axis=alt.Axis(title='Percentage')),
    color=alt.Color('Opinion:N', scale=alt.Scale(domain=['approve', 'disapprove'], range=['green', 'red']), legend=alt.Legend(title='Opinion')),
).properties(
    title='Biden Approval Rates Overtime',
    width=600,
    height=400
)

# Calculate smoothed values using loess from statsmodels
loess_smoothed = df_long.groupby(['Opinion', 'startDate']).mean().reset_index()
loess_smoothed['smoothed'] = loess_smoothed.groupby('Opinion')['Percentage'].transform(lambda x: sm.nonparametric.lowess(x, range(len(x)), frac=0.3)[:, 1])

# Altair Chart for smoothed lines
chart_lines = alt.Chart(loess_smoothed).mark_line(color='black').encode(
    x=alt.X('startDate:T', axis=alt.Axis(tickCount='month', format='%b %Y', labelAngle=-90), title='Date'),
    y=alt.Y('smoothed:Q', axis=alt.Axis(title='Percentage')),
    detail='Opinion:N'
)

# Combine scatter plot, filled circles, black smooth lines, and raw data points
combined_chart = alt.layer(chart_points, chart_lines).interactive()

# Show the combined chart
combined_chart


alt.LayerChart(...)